In [6]:

import os
import torch
from torch.utils.data import DataLoader
import pandas as pd
from transformers import BertForMaskedLM, BertTokenizer


/home/leyang.sun/.pyenv/versions/miniconda3-latest/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Load the pre-trained model state dictionary
model_state_dict = torch.load('/home/leyang.sun/BERHT/BEHRT/saved_model/BERT_Large_v1_2023-10-19')

# Load the model's configuration
model_config = BertConfig.from_pretrained('/home/leyang.sun/BERHT/BEHRT/saved_model/BERT_Large_v1_2023-10-19/config.json')

# Create an instance of the model using the configuration
model = BertForMaskedLM(config=model_config)

# Load the state dictionary into the model
model.load_state_dict(model_state_dict)

# Set the model in evaluation mode
model.eval()

TypeError: BertForMaskedLM.__init__() missing 1 required positional argument: 'config'

In [ ]:


# Load the pretrained model
pretrained_model_path = 'path_to_pretrained_model'  # Replace with the path to your pretrained model
pretrained_model = BertForMaskedLM.from_pretrained(pretrained_model_path)
pretrained_model.eval()

# Define a function to evaluate the model
def evaluate_model(model, data_loader):
    predictions = []
    labels = []

    for batch in data_loader:
        age_ids, input_ids, posi_ids, segment_ids, attMask, masked_label = batch

        with torch.no_grad():
            # Forward pass
            loss, pred, label = model(input_ids, age_ids, segment_ids, posi_ids, attention_mask=attMask, masked_lm_labels=masked_label)

            # Calculate the predicted labels
            predicted_labels = torch.argmax(pred, dim=2).cpu().numpy()

        predictions.extend(predicted_labels)
        labels.extend(label.cpu().numpy())

    return predictions, labels

# Load and evaluate the validation datasets
validation_folder = '/data/datasets/leyang.sun/BEHRT_validation/'
validation_results = []

for file_name in os.listdir(validation_folder):
    if file_name.endswith('.parquet'):
        file_path = os.path.join(validation_folder, file_name)

        # Load the validation dataset
        validation_data = pd.read_parquet(file_path)

        # Create a DataLoader for validation
        validation_dataset = MLMLoader(validation_data, token2idx, ageVocab, max_len=train_params['max_len_seq'], code='diagnosis_code')
        validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=train_params['batch_size'], shuffle=False)

        # Evaluate the model on the validation dataset
        predictions, labels = evaluate_model(pretrained_model, validation_data_loader)

        # Append the results to the validation_results list
        validation_results.append({
            'file_name': file_name,
            'predictions': predictions,
            'labels': labels
        })

# You can now analyze or display the results in the validation_results list as needed
# For example, compute evaluation metrics like precision, recall, F1-score, etc., for each validation set
